In [ ]:
import pandas as pd
import gc
import re
import os
import shutil
import nd2
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import euclidean
from scipy.ndimage import rotate, zoom
from tqdm import tqdm
import tifffile
import datetime


class SlideScannerImageProcessor:
    """
    A class to process and visualize brain histology data collected from AZ100 Multizoom Slide Scanner.
    """

    def __init__(self, default_directory):
        """
        Initialize the SlideScannerImageProcessor.

        Parameters:
        - default_directory (str): The default directory for file selection.
        """
        self.default_directory = default_directory
        self.acquisition_path  = None
        self.filepath          = None
        self.df                = None
        self.sorted_df         = None
        self.ROI_Angle_Sorter  = None
     
    
    def load_ROI_position_data(self):
        
        # extract ROI_positions from nd2 files in choosen directory
        """
        Load ROI positions from .nd2 files in the specified folder.

        Parameters:
        - folder_path: The path to the folder containing the .nd2 files.
        """
        
        # Choose a directory using the GUI
        self.__choose_directory()
        
        # Lists to store metadata
        metadata = []
        
        # Define the subfolder path
        subfolder_path = os.path.join(self.acquisition_path, '_raw_imaging_data')
        
        # Iterate over files in the subfolder
        for filename in os.listdir(subfolder_path):
            if "Region" in filename and filename.endswith(".nd2"):
                file_path = os.path.join(subfolder_path, filename)

                # Open the .nd2 file
                with nd2.ND2File(file_path) as f:
                    
                    # Extract metadata
                    all_metadata = f.unstructured_metadata()
                    dXPos = all_metadata['ImageMetadataSeqLV|0']['SLxPictureMetadata']['XPos']
                    dYPos = all_metadata['ImageMetadataSeqLV|0']['SLxPictureMetadata']['YPos']

                # Append data to metadata list
                metadata.append((filename, dXPos, dYPos))
                
                
        # Convert metadata_data to a pandas DataFrame
        self.df = pd.DataFrame(metadata, columns=['filename', 'dXPos', 'dYPos'])
        
        # Add 'ROI_position' column
        self.df['ROI_position'] = list(zip(-self.df['dXPos'], self.df['dYPos']))
    
        
    def pre_process_ROIs(self):
        """
        pre-process the ROIs if some slices were not imaged or if ROIs not containing any brain slices were scanned .
        """
        # Extract indices from filename in self.df
        self.df['cartridge'], self.df['slide'], self.df['ROI_number'] = zip(*self.df['filename'].map(self.__extract_indices_from_filename))

        # Sort DataFrame
        self.df = self.df.sort_values(by=['cartridge', 'slide', 'ROI_number']).reset_index(drop=True)

        # Apply function to each group of data based on cartridge and slide
        self.df = self.df.groupby(['cartridge', 'slide'], as_index=False).apply(self.__assign_roi_index)

        # Sort the DataFrame based on the new index and reset the overall index
        self.df = self.df.sort_values(by=['cartridge', 'slide', 'ROI_index']).reset_index(drop=True)

    
    def __choose_directory(self):
        """
        Open a GUI to choose a directory.
        """  
        # Create a Tkinter root window
        root = tk.Tk()
        # Hide the root window, as we only need it for the file dialog
        root.withdraw()

        # Open a file dialog with the primary display as the parent window
        self.acquisition_path = filedialog.askdirectory(title="Select Directory", 
                                                        initialdir=self.default_directory)
        print('Working directory: ', self.acquisition_path)

    
#     def __read_csv_as_dataframe(self):
#         """
#         Read CSV file into a DataFrame.

#         Returns:
#         - pd.DataFrame: The DataFrame containing the CSV data.
#         """
#         # Read the CSV file into a DataFrame
#         df = pd.read_csv(self.filepath)

#         # Rename 'fname' column to 'filename'
#         df.rename(columns={'fname': 'filename'}, inplace=True)

#         # Combine 'dXPos' and 'dYPos' into tuples and save in 'ROI_position'
#         df['ROI_position'] = list(zip(-df['dXPos'], df['dYPos']))

#         return df

    
    def __extract_indices_from_filename(self, filename):
        """
        Extract indices from the filename.

        Parameters:
        - filename (str): The filename to extract indices from.

        Returns:
        - tuple: A tuple containing cartridge, slide, and ROI number.
        """
        match = re.search(r"Slide(\d{1,2})-(\d{1,2})_Region(\d+)", filename)

        return tuple(map(int, match.groups())) if match else (None, None, None)
    
    
    def angle_sort_ROIs(self, rotation_angle, cartridge, slide, origin_index):
        """
        Sort ROIs based on angle and distance from origin.

        Parameters:
        - rotation_angle (float): The angle for sorting.
        - cartridge (int): The cartridge number.
        - slide (int): The slide number.
        - origin_index (int): The index of the origin point.

        Returns:
        - pd.DataFrame: The sorted DataFrame.
        """
        self.ROI_Angle_Sorter = ROI_Angle_Sorter(rotation_angle, cartridge, slide, origin_index)
       
        if self.df is not None:
            return self.ROI_Angle_Sorter.angle_sort_ROIs(self.df)
        else:
            print("Data not load.")
            return None    
            

    def hist_sort_ROIs(self, cartridge, slide, grid_num_bins_x=4, grid_num_bins_y=2):
        """
        Sorts the Regions of Interest (ROIs) based on a histogram.
         
        Args:
             - cartridge (int): The cartridge number.
             - slide (int): The slide number.
             - grid_num_bins_x (int): max number of slices on slide along x-axis
             - grid_num_bins_y (int): max number of slices on slide along y-axis
        
        Returns:
            DataFrame: The sorted DataFrame containing ROI data.
        """

        self.cartridge = cartridge
        self.slide     = slide
        
        # Filter DataFrame based on cartridge and slide
        self.sorted_df = self.df[(self.df['cartridge'] == cartridge) & 
                                 (self.df['slide']     == slide)].copy()
        
        # Extract ROI positions
        self.ROI_positions = np.array(self.sorted_df['ROI_position'].tolist())
    
        # define range for histogram also avoiding that a ROI coordinate lies on the bin edge
        x_min, x_max = np.min( self.ROI_positions[:, 0]) -1, np.max( self.ROI_positions[:, 0]) +1
        y_min, y_max = np.min( self.ROI_positions[:, 1]) -1, np.max( self.ROI_positions[:, 1]) +1
    
        # Create a 2D histogram
        H, xedges, yedges = np.histogram2d(self.ROI_positions[:, 0], self.ROI_positions[:, 1], 
                                           bins  = (grid_num_bins_x, grid_num_bins_y), 
                                           range = [[x_min, x_max], [y_min, y_max]])
    
        # Check if any element in H is not 0 or 1
        if np.any((H != 0) & (H != 1)):
            raise ValueError("Invalid histogram values, e.g. more than one point may have been found per bin.")
    
        # Reverse the order of xedges (such that the first bin in the xedges list is the bin in the lower-right corner and
        # not the lower-left corner), if the slice origin on the slide ever changes this is where changes need to be made
        x_indices = np.digitize(self.ROI_positions[:, 0], xedges[::-1]) - 1
        y_indices = np.digitize(self.ROI_positions[:, 1], yedges) - 1
    
        # these indices correspond to the position of the corresponding bins in the flattened histogram H.
        indices_reshaped = np.ravel_multi_index((y_indices, x_indices), (grid_num_bins_y, grid_num_bins_x))
    
        # reorder such that empty histogram bins are not counted 
        sorted_indices                     = np.argsort(indices_reshaped)
        sorted_index_array                 = np.zeros_like(indices_reshaped)
        sorted_index_array[sorted_indices] = np.arange(len(indices_reshaped))
    
        # Add sorted indices to sorted_df
        self.sorted_df['resorted_ROI_index'] = sorted_index_array
        
        # Generate new filenames based on sorting
        self.sorted_df['new_filename'] = self.sorted_df.apply(self.__generate_new_filename, axis=1)
                
        self.sorted_ROI_positions = self.sorted_df['ROI_position'].tolist()  
    
    
    def __generate_new_filename(self, row):
        """
        Generate a new filename based on sorting.
        
        Args:
            row (Series): The row containing filename data.
        
        Returns:
            str: The new filename based on sorting.
        """
        # Extract cartridge, slide, and scanned ROI index from filename
        match = re.search(r"(.*Slide\d+-\d+_Region)(\d+)(_.*)", row['filename'])
        if match:
            prefix = match.group(1)
            scanned_roi = match.group(2)
            suffix = match.group(3)
            
            # Replace scanned ROI index with resorted ROI index while keeping leading zeros
            new_roi_index = str(row['resorted_ROI_index']).zfill(len(scanned_roi))
            new_filename = f"{prefix}{new_roi_index}{suffix}"
            return new_filename
        else:
            return row['filename']

        
    def rename_original_nd2_images(self):
        """
        """

        # Iterate through sorted_df and copy files to new folder
        for _, row in tqdm(self.ROI_Angle_Sorter.sorted_df.iterrows(), total=self.ROI_Angle_Sorter.sorted_df.shape[0], desc="Copying ROI images"):
            source_file      = os.path.join(self.acquisition_path, row['filename'])
            destination_file = os.path.join(self.acquisition_path, row['new_filename'])

            shutil.move(source_file, destination_file)

        print('Completed!')

    
    def copy_sorted_nd2s_to_target_folder(self, target_folder = "_processed"):
        """
        Copy sorted nd2 images to a custom named folder in the given directory.

        Parameters:
        - new_folder_name (str, optional): Name of the folder. Defaults to "_processed".

        Returns:
        - None
        """
        
        # Define the source subfolder path
        subfolder_path = os.path.join(self.acquisition_path, '_raw_imaging_data')
    
        # Create new folder
        target_folder_path = self.__create_target_folder(self.acquisition_path, target_folder)

        # Iterate through sorted_df and copy files to new folder
        for _, row in tqdm(self.sorted_df.iterrows(), total=self.sorted_df.shape[0], desc="Copying ROI images"):
            source_file      = os.path.join(subfolder_path, row['filename'])
            destination_file = os.path.join(target_folder_path, row['new_filename'])

            # Check if the destination file already exists
            if os.path.exists(destination_file):
                print(f"The file {destination_file} already exists. Skipping this file.\n")
                continue
            
            shutil.copy(source_file, destination_file)

        print('Completed!')


    def __create_target_folder(self, default_directory, target_folder):
        """
        Create a custom named new folder in the given default directory if it doesn't exist.

        Parameters:
        - default_directory (str): The directory in which to create the new folder.
        - target_folder (str): Name of the target folder.

        Returns:
        - str: The path to the created or existing target folder.
        """
        target_folder_path = os.path.join(default_directory, target_folder)
        
        if not os.path.exists(target_folder_path):
            os.makedirs(target_folder_path)
        
        return target_folder_path
            

    def __assign_roi_index(self, group):
        """
        Assign an index within each cartridge-slide combination.

        Parameters:
        - group (pd.DataFrame): A group of data.

        Returns:
        - pd.DataFrame: The group with ROI index assigned.
        """
        group['ROI_index'] = range(len(group))
        
        return group
    
    
    def plot_scanned_slide_data(self):
        """
        Plot the scanned slide data.
        """
        # Get unique combinations of cartridges and slides
        unique_combinations = self.df[['cartridge', 'slide']].drop_duplicates()

        # Calculate the number of plots and the number of rows and columns
        num_plots = len(unique_combinations)
        num_cols = min(num_plots, 3)
        num_rows = -(-num_plots // num_cols)  # Ceiling division to get the number of rows

        # Calculate the figure size based on the number of rows
        figsize = (15, 3 * num_rows) if num_plots > 1 else (8, 3)

        # Create subplots
        fig, axes = plt.subplots(num_rows, num_cols, figsize=figsize)

        # Check if axes is an instance of np.ndarray
        if not isinstance(axes, np.ndarray):
            axes = [axes]
        else:
            axes = axes.ravel()

        # Initialize variables to store maximum x and y values
        max_x_value = float('-inf')
        min_x_value = float('inf')
        max_y_value = float('-inf')
        min_y_value = float('inf')

        for ii, (ax, (cartridge, slide)) in enumerate(zip(axes, unique_combinations.itertuples(index=False))):
            ax.set_title(f"Cartridge {cartridge}, Slide {slide}")
            ax.set_xlabel('X Position')

            # Set Y Position label only on the leftmost graphs
            if ii % num_cols == 0:
                ax.set_ylabel('Y Position')

            # Filter DataFrame for the current combination
            filtered_df = self.df[(self.df['cartridge'] == cartridge) & (self.df['slide'] == slide)]

            # Extract x and y values
            x_values, y_values = zip(*filtered_df['ROI_position'])

            # Update max and min x and y values
            max_x_value = max(max_x_value, max(x_values))
            min_x_value = min(min_x_value, min(x_values))
            max_y_value = max(max_y_value, max(y_values))
            min_y_value = min(min_y_value, min(y_values))

            # Plot ROI positions and annotate with region index
            for _, row in filtered_df.iterrows():
                ax.scatter(row['ROI_position'][0], row['ROI_position'][1], label=row['filename'])
                ax.annotate(row['ROI_number'], row['ROI_position'], textcoords="offset points", xytext=(0, 10), ha='center')

            # Adjust font size of both x and y axis tick labels
            ax.tick_params(axis='both', labelsize=8)

        # Calculate margins based on the maximum and minimum values across all subplots
        x_margin = (max_x_value - min_x_value) * 0.2
        y_margin = (max_y_value - min_y_value) * 0.5

        # Adjust limits for all subplots
        for ax in axes:
            ax.set_xlim(min_x_value - x_margin, max_x_value + x_margin)
            ax.set_ylim(min_y_value - y_margin, max_y_value + y_margin)

        # Turn off extra subplots if any
        for ax in axes[num_plots:]:
            ax.axis('off')

        # Add title and adjust layout
        plt.suptitle(os.path.basename(self.acquisition_path), fontsize=16)
        plt.tight_layout()
        
        plt.show()

        
    def plot_angle_sorted_ROI_data(self):
        
        # Plotting
        fig, axs = plt.subplots(2, 2, figsize=(10, 5))

        # Scanned ROIs and indices
        
        colors = []  # Initialize an empty list to store the colors
        
        for ii, (x, y) in enumerate(self.ROI_Angle_Sorter.ROI_positions):
            scatter = axs[0, 0].scatter(x, y, s=50)
            axs[0, 0].annotate(str(self.ROI_Angle_Sorter.filtered_df['ROI_index'].iloc[ii]), (x, y), 
                               textcoords="offset points", 
                               xytext=(5,5), 
                               ha='center', 
                               fontsize=10)
            colors.append(scatter.get_facecolor()[0])  # Append the color of the scatter plot to the list
            
        axs[0, 0].scatter(self.ROI_Angle_Sorter.origin_x, self.ROI_Angle_Sorter.origin_y, color='black', s=120)
        axs[0, 0].set_title('Scanned ROIs and indices')
        
        
        # Increase x-axis and y-axis limits by 0.1x
        xlim = axs[0, 0].get_xlim()
        ylim = axs[0, 0].get_ylim()
        axs[0, 0].set_xlim(xlim[0] + 0.1*xlim[0], xlim[1] - 0.1*xlim[1])
        axs[0, 0].set_ylim(ylim[0] - 0.1*ylim[0], ylim[1] + 0.1*ylim[1])
        
        
        # Transformed ROI positions
        for ii, (x, y) in enumerate(self.ROI_Angle_Sorter.transformed_ROI_positions):
            axs[0, 1].scatter(x, y, s=50)
            axs[0, 1].annotate(str(self.ROI_Angle_Sorter.filtered_df['ROI_index'].iloc[ii]), (x, y), 
                               textcoords="offset points", 
                               xytext=(5,5), 
                               ha='center', 
                               fontsize=10)
        axs[0, 1].scatter(0, 0, color='black', s=120)
        axs[0, 1].set_title('Transformed ROIs (affine rotation around origin)')

        # Increase x-axis and y-axis limits
        xlim = axs[0, 1].get_xlim()
        ylim = axs[0, 1].get_ylim()
        axs[0, 1].set_xlim(xlim[0] - 5000, xlim[1] + 5000)
        axs[0, 1].set_ylim(ylim[0] - 5000, ylim[1] + 5000)
        

        # Angle vs. Distance from Origin
        for ii, (x, y) in enumerate(zip(self.ROI_Angle_Sorter.ROI_angles, self.ROI_Angle_Sorter.ROI_distances)):
            axs[1, 0].scatter(x, y, s=50)
            axs[1, 0].annotate(str(self.ROI_Angle_Sorter.filtered_df['ROI_index'].iloc[ii]), (x, y), 
                               textcoords="offset points", 
                               xytext=(5,5), 
                               ha='center', 
                               fontsize=10)
        axs[1, 0].scatter(0, 0, color='black', label='New Origin', s=120)
        axs[1, 0].set_xlabel("Angle (degrees)")
        axs[1, 0].set_ylabel("Distance (units)")
        axs[1, 0].set_title("Angle vs. Distance from Origin ROI")

        # Increase x-axis and y-axis limits
        xlim = axs[1, 0].get_xlim()
        ylim = axs[1, 0].get_ylim()
        axs[1, 0].set_xlim(xlim[0] - 10, xlim[1] + 10)
        axs[1, 0].set_ylim(ylim[0] - 5000, ylim[1] + 12000)
        
        
        # Sorted ROI indices
        sorted_colors = [colors[ii] for ii in self.ROI_Angle_Sorter.sorted_df['ROI_index']]  # Reorder the colors
        for ii, (x, y) in enumerate(self.ROI_Angle_Sorter.sorted_ROI_positions):
            axs[1, 1].scatter(x, y, s=50, color=sorted_colors[ii])  # Use the reordered colors
            axs[1, 1].annotate(str(self.ROI_Angle_Sorter.sorted_df['resorted_ROI_index'].iloc[ii]), (x, y), 
                               textcoords="offset points", 
                               xytext=(5,5), 
                               ha='center', 
                               fontsize=10)
            
        axs[1, 1].scatter(self.ROI_Angle_Sorter.origin_x, self.ROI_Angle_Sorter.origin_y, color='black', s=120)
        axs[1, 1].set_title('Sorted ROI indices')
        
        # Increase x-axis and y-axis limits by 0.1x
        xlim = axs[1, 1].get_xlim()
        ylim = axs[1, 1].get_ylim()
        axs[1, 1].set_xlim(xlim[0] + 0.1*xlim[0], xlim[1] - 0.1*xlim[1])
        axs[1, 1].set_ylim(ylim[0] - 0.1*ylim[0], ylim[1] + 0.1*ylim[1])

        plt.suptitle(f"{os.path.basename(self.acquisition_path)} \nCartridge {self.ROI_Angle_Sorter.cartridge}, Slide {self.ROI_Angle_Sorter.slide}")
        plt.tight_layout()
        plt.show()


    def plot_hist_sorted_ROI_data(self):
        
        # Plotting
        fig, axs = plt.subplots(1, 2, figsize=(10, 3))

        # Scanned ROIs and indices
        colors = []  # Initialize an empty list to store the colors
        
        for ii, (x, y) in enumerate(self.ROI_positions):
            scatter = axs[0].scatter(x, y, s=50)
            axs[0].annotate(str(self.sorted_df['ROI_number'].iloc[ii]), (x, y), 
                               textcoords="offset points", 
                               xytext=(5,5), 
                               ha='center', 
                               fontsize=10)
            colors.append(scatter.get_facecolor()[0])  # Append the color of the scatter plot to the list
            
        axs[0].set_title('Scanned ROIs and indices')
        
        # Increase x-axis and y-axis limits by 0.1x
        xlim = axs[0].get_xlim()
        ylim = axs[0].get_ylim()
        axs[0].set_xlim(xlim[0] + 0.1*xlim[0], xlim[1] - 0.1*xlim[1])
        axs[0].set_ylim(ylim[0] - 0.1*ylim[0], ylim[1] + 0.1*ylim[1])
        
        # Sorted ROI indices
        sorted_colors = [colors[ii] for ii in self.sorted_df['ROI_index']]  # Reorder the colors
        for ii, (x, y) in enumerate(self.sorted_ROI_positions):
            axs[1].scatter(x, y, s=50, color=sorted_colors[ii])  # Use the reordered colors
            axs[1].annotate(str(self.sorted_df['resorted_ROI_index'].iloc[ii]), (x, y), 
                               textcoords="offset points", 
                               xytext=(5,5), 
                               ha='center', 
                               fontsize=10)
            
        axs[1].set_title('Sorted ROI indices')
        
        # Increase x-axis and y-axis limits by 0.1x
        xlim = axs[1].get_xlim()
        ylim = axs[1].get_ylim()
        axs[1].set_xlim(xlim[0] + 0.1*xlim[0], xlim[1] - 0.1*xlim[1])
        axs[1].set_ylim(ylim[0] - 0.1*ylim[0], ylim[1] + 0.1*ylim[1])

        plt.suptitle(f"{os.path.basename(self.acquisition_path)} \nCartridge {self.cartridge}, Slide {self.slide}")
        plt.tight_layout()
        plt.show()
    
    
    def convert_nd2_to_tif(self, rotation_angle = 180, scale_factor = 0.5, target_folder = '_sorted_tifs'):
        
        # Create a root window and hide it
        root = tk.Tk()
        root.withdraw()
        
        # Open a file dialog with the primary display as the parent window
        folder_path = filedialog.askdirectory(title="Select Directory", 
                                              initialdir=self.acquisition_path)
        
        # Define the subfolder path
        subfolder_path = os.path.join(folder_path, '_sorted_nd2s')

        # Create a new directory for the output files
        output_dir = os.path.join(folder_path, target_folder)
        os.makedirs(output_dir, exist_ok=True)
        
        # Get a list of all .nd2 files in the selected subfolder
        nd2_files = [ff for ff in os.listdir(subfolder_path) if ff.endswith(".nd2") and 'Region' in ff]

        print(f"convert_nd2_to_tif started at: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}")

        # Iterate over all files in the selected folder
        for filename in tqdm(nd2_files, desc="Converting files"):

            # Construct the full file path
            file_path = os.path.join(subfolder_path, filename)
            
            output_file_path = os.path.join(output_dir, filename.replace('.nd2', '.tif'))

            # Check if the output file already exists
            if os.path.exists(output_file_path):
                print(f"The converted tif-file {output_file_path} already exists. Skipping this file.\n")
                continue

            # Read the nd2 file
            with nd2.ND2File(file_path) as image:
                # img_array has shape (c,y,x)
                img_array = image.asarray()


                #----------------Image Processing-----------------

                # Rotate the image by a default 180 degrees using the y and x axes
                img_array = rotate(img_array, rotation_angle, axes=(1,2))

                # Rescale the image along the x and y axis
                img_array = zoom(img_array, (1, scale_factor, scale_factor))

                #----------------Image Processing-----------------

                # Get the original scale
                original_pixel_size = image.voxel_size()[0]

                # Write the tif file to the new directory
                tifffile.imwrite(output_file_path, 
                                 img_array, 
                                 imagej     = True, 
                                 resolution = (1/original_pixel_size * scale_factor, 1/original_pixel_size * scale_factor), 
                                 metadata   = {'spacing': 1,
                                               'unit': 'um',
                                              }
                                )     

            # Delete the variables to free up memory
            del img_array

            # Run the garbage collector
            gc.collect()

        print("Conversion completed.")
            
        
    def __calculate_distance(self, miniRuby_df):
    
        # Initialize variables
        increments = []
        tmp        = []

        # Loop through the DataFrame
        for index, row in miniRuby_df.iterrows():
            if row['slice_thickness_microns'] == 1000:
                increments.append(tmp)
                tmp = []
            else:
                tmp.append((index, row))

        # Add the last increment
        increments.append(tmp)

        # Calculate the distance for each increment
        for ii, increment in enumerate(increments, start=0):
            try:
                start_index = next(index for index, row in increment if row['target'] == -1)
                end_index   = next(index for index, row in increment if row['target'] == 1)

                # distances where the target is further posterior than the injection site are positive, 
                # otherwise negative 
                if start_index < end_index:
                    factor =  1
                else:
                    factor = -1

                # Switching start and end
                start_index, end_index = min(start_index, end_index), max(start_index, end_index)

                distance = factor * miniRuby_df.loc[start_index:end_index - 1, 'slice_thickness_microns'].sum()

                print(f"Area {ii}: Distance(inj.,target) = {distance} microns")

            except StopIteration:
                print(f"Area {ii}: Target not found")


    def plot_focal_injection_profile(self):
        """
        This function plots the focal injection profile from a CSV file.
        The CSV file is selected by the user from a directory.
        The function plots the median values and distance from the brain surface in microns.
        """

        # Initialize a Tkinter root window and hide it
        root = tk.Tk()
        root.withdraw()

        # Ask the user to select a directory
        folder_selected = filedialog.askdirectory(title="Select Directory", 
                                                  initialdir=self.default_directory)

        # Define the CSV filename substring to look for
        csv_filename_substring = '_miniRuby_intensity'

        # Get a list of all CSV files in the selected directory that contain the substring
        csv_files = [filename for filename in os.listdir(folder_selected) if csv_filename_substring in filename]

        # If no matching CSV files are found, print a message and return
        if not csv_files:
            print(f"No CSV file with '{csv_filename_substring}' found in the selected directory.")
            return

        # Load the first matching CSV file into a pandas DataFrame
        csv_filepath = os.path.join(folder_selected, csv_files[0])
        df           = pd.read_csv(csv_filepath, index_col=0)

        # Extract the median values and calculate the x-values
        median   = df['Median']
        x_values = df['slice_thickness_microns'].cumsum() - df['slice_thickness_microns'].iloc[0]

        # calculate the distance in microns between
        self.__calculate_distance(df)

        # Create a plot of the median values
        fig, ax1 = plt.subplots(figsize=(10, 5))

        # Check if 'filename' is equal to 'predicted_slide'
        mask_predicted = df['filename'] == 'predicted_slide'

        # Plot with filled circle markers for non-'predicted_slide' rows
        ax1.plot(df.index[~mask_predicted], median[~mask_predicted], 
                 color='C1', marker='o', markersize=10, linestyle='', label='Median')

        # Plot with empty circle markers only for 'predicted_slide'
        ax1.plot(df.index[mask_predicted], median[mask_predicted], 
                 color='C1', marker='o', markersize=10, linestyle='', 
                 fillstyle='none', label='Predicted Slide')

        ax1.plot(df.index, median, color='C1', linestyle='-')


        # Annotate points where 'target' is 1 or -1
        for ii in df.index:
            if df.loc[ii, 'target'] == 1:
                ax1.annotate('* target', (ii, median[ii]), xytext=(0, 10), textcoords='offset points')
            elif df.loc[ii, 'target'] == -1:
                ax1.annotate('* inj.', (ii, median[ii]), xytext=(0, 10), textcoords='offset points')

        # Increase the max y-limit by 10%
        ylim = ax1.get_ylim()
        ax1.set_ylim((ylim[0], ylim[1] + 0.1 * ylim[1]))

        # Set the x-axis labels and title
        ax1.set_xticks(np.arange(len(median)))
        ax1.set_xlabel('Slice Index')
        ax1.set_ylabel('miniRuby median intensity value', color='C1', fontdict=dict(weight='bold'))

        # Create a second x-axis for the position along the AP-Axis
        ax2 = ax1.twiny()
        ax2.set_xlim(ax1.get_xlim())
        ax2.set_xticks(np.arange(len(median)))
        ax2.set_xticklabels(x_values.values, fontsize=6)
        ax2.set_xlabel('Position Along AP-Axis (Microns)')

        # Create a second y-axis for the distance from the brain surface
        ax3 = ax1.twinx()
        ax3.plot(df.index, df.distance_to_surface_microns, marker='o', color='C0')
        ax3.set_ylabel('Distance from Brain Surface (Microns)', color='C0', fontdict=dict(weight='bold'))
        ax3.set_ylim(2000, 5000)

        # Set the plot title and show the grid
        plt.suptitle(os.path.basename(folder_selected), fontsize=12)
        plt.grid(True)

        # Adjust the spacing between the two x-axes and show the plot
        plt.tight_layout()
        plt.show()
        
    

class ROI_Angle_Sorter:
    """
    A class for sorting Regions of Interest (ROIs) based on the angle and distance from origin ROI.
    
    Attributes:
        rotation_angle (float): The rotation angle for sorting ROIs distributed in a grid on the slide.
        cartridge (str): The slide scanner cartridge identifier for the ROIs.
        slide (str): The slide identifier for the ROIs.
        origin_index (int): The index of the origin ROI for sorting.
    """

    def __init__(self, rotation_angle, cartridge, slide, origin_index):
        """
        Initialize the ROI_Angle_Sorter object with rotation angle, cartridge, slide, and origin index.
        
        Args:
            rotation_angle (float): The rotation angle for sorting ROIs.
            cartridge (str): The cartridge identifier for the ROIs.
            slide (str): The slide identifier for the ROIs.
            origin_index (int): The index of the origin ROI for sorting.
        """
        self.rotation_angle = rotation_angle
        self.cartridge      = cartridge
        self.slide          = slide
        self.origin_index   = origin_index

        
    def angle_sort_ROIs(self, df):
        """
        Sorts the Regions of Interest (ROIs) based on the angle and distance from origin ROI.
        
        Args:
            df (DataFrame): The DataFrame containing ROI data.
            
        Returns:
            DataFrame: The sorted DataFrame containing ROI data.
        """
        # Filter DataFrame based on cartridge and slide
        self.filtered_df = df[(df['cartridge'] == self.cartridge) & 
                              (df['slide'] == self.slide)].copy()
        
        # Extract ROI positions
        self.ROI_positions = self.filtered_df['ROI_position'].tolist()
        
        # Calculate origin coordinates and transform ROI positions
        self.origin_x, self.origin_y = self.ROI_positions[self.origin_index]
        rotation_angle_rad = np.deg2rad(self.rotation_angle)
        self.transformed_ROI_positions = [(self.__rotate_point(x - self.origin_x, 
                                                               y - self.origin_y, 
                                                               rotation_angle_rad)) 
                                           for x, y in self.ROI_positions]

        # Calculate angles and distances from the origin
        self.ROI_angles, self.ROI_distances = zip(*[(self.__calculate_angle((0, 0), point, [0, 1]), 
                                                     self.__calculate_distance((0, 0), point))
                                                     for point in self.transformed_ROI_positions])
        
        # Add angle and distance columns to DataFrame
        self.filtered_df['angle_to_origin'] = self.ROI_angles
        self.filtered_df['distance_to_origin'] = self.ROI_distances

        # Apply custom sorting function based on angle and distance
        self.filtered_df['sorting_key'] = self.filtered_df.apply(self.__custom_sort, axis=1)
        self.sorted_df = self.filtered_df.sort_values(by='sorting_key').drop(columns='sorting_key')
        self.sorted_df['resorted_ROI_index'] = range(len(self.sorted_df))
        
        # Generate new filenames based on sorting
        self.sorted_df['new_filename'] = self.sorted_df.apply(self.__generate_new_filename, axis=1)
                
        self.sorted_ROI_positions = self.sorted_df['ROI_position'].tolist()  

        return self.sorted_df
    
    
    def __rotate_point(self, x, y, rotation_angle):
        """
        Rotate a point by a given angle.
        
        Args:
            x (float): The x-coordinate of the point.
            y (float): The y-coordinate of the point.
            rotation_angle (float): The angle of rotation in radians.
        
        Returns:
            tuple: The rotated coordinates of the point.
        """
        new_x = x * np.cos(rotation_angle) - y * np.sin(rotation_angle)
        new_y = x * np.sin(rotation_angle) + y * np.cos(rotation_angle)
        return new_x, new_y

    
    def __calculate_distance(self, point1, point2):
        """
        Calculate the Euclidean distance between two points.
        
        Args:
            point1 (tuple): The coordinates of the first point.
            point2 (tuple): The coordinates of the second point.
        
        Returns:
            float: The Euclidean distance between the two points.
        """
        return euclidean(point1, point2)

    
    def __calculate_angle(self, origin, point, ref_vector):
        """
        Calculate the angle between a reference vector and a point.
        
        Args:
            origin (tuple): The coordinates of the origin point.
            point (tuple): The coordinates of the point.
            ref_vector (list): The reference vector.
        
        Returns:
            float: The angle in degrees between the reference vector and the point.
        """
        if point[0] == origin[0] and point[1] == origin[1]:
            return 0.0
        dx, dy = point[0] - origin[0], point[1] - origin[1]
        angle = np.degrees(np.arctan2(dy, dx))
        return angle
    
    
    def __custom_sort(self, row):
        """
        Custom sorting function based on angle and distance from origin ROI.
        
        Args:
            row (Series): The row containing angle and distance data.
        
        Returns:
            tuple: The sorting key based on angle and distance.
        """
        angle = row['angle_to_origin']
        distance = row['distance_to_origin']
        if angle >= 0:
            return (1, distance)
        else:
            return (2, angle)
        
        
    def __generate_new_filename(self, row):
        """
        Generate a new filename based on sorting.
        
        Args:
            row (Series): The row containing filename data.
        
        Returns:
            str: The new filename based on sorting.
        """
        # Extract cartridge, slide, and scanned ROI index from filename
        match = re.search(r"(.*Slide\d+-\d+_Region)(\d+)(_.*)", row['filename'])
        if match:
            prefix = match.group(1)
            scanned_roi = match.group(2)
            suffix = match.group(3)
            
            # Replace scanned ROI index with resorted ROI index while keeping leading zeros
            new_roi_index = str(row['resorted_ROI_index']).zfill(len(scanned_roi))
            new_filename = f"{prefix}{new_roi_index}{suffix}"
            return new_filename
        else:
            return row['filename']




In [ ]:
# load SlideScannerImageProcessor
default_directory = ""
processor         = SlideScannerImageProcessor(default_directory)

In [ ]:
# load ROI-Positions from nd2 files, pre-process and plot the slice positions on the slide
processor.load_ROI_position_data()
processor.pre_process_ROIs()
processor.plot_scanned_slide_data()

In [ ]:
processor.hist_sort_ROIs(cartridge=1, slide=5, grid_num_bins_x=4, grid_num_bins_y=2)
processor.plot_hist_sorted_ROI_data()

In [ ]:
# copy to new folder or rename the original nd2 files in place
processor.copy_sorted_nd2s_to_target_folder(target_folder = "_sorted_nd2s")
#processor.rename_original_nd2_images()

In [ ]:
# convert nd2s to tiffs, rotate and scale
processor.convert_nd2_to_tif(rotation_angle = 180, scale_factor = 0.5, target_folder = "_sorted_tifs")

In [ ]:
# plot focal injection profiles (if existing)
processor.plot_focal_injection_profile()